<a href="https://colab.research.google.com/github/burakbeyazit/grad-project/blob/main/Bitirme_Projem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Data Preprocessing...

Dropping rows which contains NaN or empty values



In [ ]:
import pandas as pd

file_path = '/content/drive/MyDrive/Flight_Datas/Flight_Points_Actual_20220601_20220630.csv'

df = pd.read_csv(file_path)

# Ön temizlik: eksik verileri sil
df_clean = df.dropna().copy()

# Koordinat düzeltme fonksiyonu
def fix_coordinate(value):
    try:
        if pd.isna(value):
            return None
        if isinstance(value, str) and value.startswith("."):
            value = "0" + value
        value = float(value)
        if abs(value) > 180:
            value /= 100000
        return round(value, 6)
    except:
        return None

# Latitude ve Longitude sütunlarını temizle
df_clean["Latitude"] = df_clean["Latitude"].apply(fix_coordinate)
df_clean["Longitude"] = df_clean["Longitude"].apply(fix_coordinate)

# Hatalı koordinatları tamamen sil
df_clean = df_clean.dropna(subset=["Latitude", "Longitude"])

df_clean.to_csv(file_path, index=False)

print("Dosya eksiksiz ve temiz olarak güncellendi.")


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [2]:
!pip install netCDF4 h5netcdf xarray

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.5 MB/s eta 0:00:00


In [5]:


import pandas as pd

filed_csv  = "/content/drive/MyDrive/Flight_Datas/Flight_Points_Filed_20220601_20220630.csv"
actual_csv = "/content/drive/MyDrive/Flight_Datas/Flight_Points_Actual_20220601_20220630.csv"
nc_path    = "/content/drive/MyDrive/Flight_Datas/data_stream-oper_stepType-instant.nc"
flights_meta = '/content/drive/MyDrive/Flight_Datas/Flights_20220601_20220630.csv'

df_meta   = pd.read_csv(flights_meta)

df_filed  = pd.read_csv(filed_csv)
df_actual = pd.read_csv(actual_csv)

In [3]:
import pandas as pd

cutoff = pd.Timestamp("2022-06-07 23:00:00")
grid_res = 0.5
def preprocess(df):
    df = df.copy()
    df["Time Over"] = pd.to_datetime(df["Time Over"], dayfirst=True)
    df = df[df["Time Over"] <= cutoff]
    df["TimeHour"] = df["Time Over"].dt.floor("h")
    df["Lat_grid"] = (df["Latitude"] / grid_res).round() * grid_res
    df["Lon_grid"] = (df["Longitude"] / grid_res).round() * grid_res
    return df

df_filed = preprocess(df_filed)
df_actual = preprocess(df_actual)

In [4]:
import pandas as pd
import xarray as xr
import numpy as np
from tqdm import tqdm

# === Dosya yolları ===
filed_csv  = "/content/drive/MyDrive/Flight_Datas/Flight_Points_Filed_20220601_20220630.csv"
actual_csv = "/content/drive/MyDrive/Flight_Datas/Flight_Points_Actual_20220601_20220630.csv"
nc_path    = "/content/drive/MyDrive/Flight_Datas/data_stream-oper_stepType-instant.nc"

# === Ayarlar ===
cutoff = pd.Timestamp("2022-06-07 23:00:00")
grid_res = 0.5


# === 2) Benzersiz noktalar ===
combined = pd.concat([
    df_filed[["Lat_grid", "Lon_grid", "TimeHour"]],
    df_actual[["Lat_grid", "Lon_grid", "TimeHour"]]
])
unique_pts = combined.drop_duplicates().reset_index(drop=True)

# === 3) NetCDF aç ===
ds = xr.open_dataset(nc_path).load()

# === 4) Hava durumu verisi cache ===
weather_cache = {}
for _, row in tqdm(unique_pts.iterrows(), total=len(unique_pts), desc="⛅ Weather cache"):
    lat, lon, time = row["Lat_grid"], row["Lon_grid"], row["TimeHour"]
    key = (lat, lon, time)
    try:
        pt = ds.sel(valid_time=np.datetime64(time), latitude=lat, longitude=lon, method="nearest")
        weather_cache[key] = {
            "Temp_C": float(pt["t2m"].values) - 273.15,
            "DewPoint_C": float(pt["d2m"].values) - 273.15,
            "U_Wind_10m": float(pt["u10"].values),
            "V_Wind_10m": float(pt["v10"].values),
            "MSL_Pressure_hPa": float(pt["msl"].values) / 100,
            "Surface_Pressure_hPa": float(pt["sp"].values) / 100,
            "CloudBase_m": float(pt["cbh"].values),
            "LowCloud": float(pt["lcc"].values),
            "TotalCloud": float(pt["tcc"].values),
        }
    except:
        weather_cache[key] = {k: None for k in [
            "Temp_C", "DewPoint_C", "U_Wind_10m", "V_Wind_10m",
            "MSL_Pressure_hPa", "Surface_Pressure_hPa",
            "CloudBase_m", "LowCloud", "TotalCloud"
        ]}

# === 5) DataFrame’e çevir ===


⛅ Weather cache: 100%|██████████| 1415476/1415476 [19:45<00:00, 1194.02it/s]


KeyError: 'index'

In [5]:
wc_df = pd.DataFrame.from_dict(weather_cache, orient="index")
wc_df.index = pd.MultiIndex.from_tuples(wc_df.index, names=["Lat_grid", "Lon_grid", "TimeHour"])
wc_df.reset_index(inplace=True)


# === 6) Merge ile dosyalara ekle ===
df_filed  = df_filed.merge(wc_df, on=["Lat_grid", "Lon_grid", "TimeHour"], how="left")
df_actual = df_actual.merge(wc_df, on=["Lat_grid", "Lon_grid", "TimeHour"], how="left")

# === 7) Temizlik ve kayıt ===
df_filed.drop(columns=["Lat_grid", "Lon_grid", "TimeHour"], inplace=True)
df_actual.drop(columns=["Lat_grid", "Lon_grid", "TimeHour"], inplace=True)

df_filed.to_csv(filed_csv, index=False)
df_actual.to_csv(actual_csv, index=False)

print("✅ Filed ve Actual verilerine hava durumu başarıyla eklendi.")


✅ Filed ve Actual verilerine hava durumu başarıyla eklendi.


Adding Destination and Source point informations on route logs

This project compares planned and actual flight paths using a machine learning model.
We trained a Random Forest model to predict how much the actual flight deviates from the planned route.
We used 300 flights and split the data into 80% training and 20% testing.
The model works well, with an R² score of 0.959 and low prediction error.
We also visualized the real vs predicted deviations, and most of the predictions are very close to the actual values.

In [4]:
print(df_meta.columns.tolist())


['ECTRL ID', 'ADEP', 'ADEP Latitude', 'ADEP Longitude', 'ADES', 'ADES Latitude', 'ADES Longitude', 'FILED OFF BLOCK TIME', 'FILED ARRIVAL TIME', 'ACTUAL OFF BLOCK TIME', 'ACTUAL ARRIVAL TIME', 'AC Type', 'AC Operator', 'AC Registration', 'ICAO Flight Type', 'STATFOR Market Segment', 'Requested FL', 'Actual Distance Flown (nm)']


In [11]:
# 1) Meta’dan sadece ECTRL ID, ADEP, ADES sütunlarını al
meta_subset = df_meta[["ECTRL ID", "ADEP", "ADES"]]

# 2) actual ve filed’e merge ederken suffix vermeye gerek yok
df_actual = df_actual.merge(
    meta_subset,
    on="ECTRL ID",
    how="left"
)
df_filed = df_filed.merge(
    meta_subset,
    on="ECTRL ID",
    how="left"
)

# 3) Eğer önceden yaptığın merge’lerde _x/_y kalmışsa onları temizle
#    (aşağıdaki iki satırı buna göre düzenle)
for df in (df_actual, df_filed):
    # eğer _x/_y kolonları varsa drop et
    for col in ["ADEP_x","ADES_x","ADEP_y","ADES_y"]:
        if col in df.columns:
            df.drop(columns=col, inplace=True)

# 4) Artık df_actual ve df_filed’da sadece tek bir ADEP, ADES sütunu var
print(df_actual.head())
print(df_filed.head())


    ECTRL ID  Sequence Number           Time Over  Flight Level  Latitude  \
0  252119478                0 2022-05-31 23:50:00             0  51.88500   
1  252119478                1 2022-06-01 00:02:00             0  51.88500   
2  252119478                2 2022-06-01 00:03:25            29  51.83500   
3  252119478                3 2022-06-01 00:04:15            50  51.85833   
4  252119478                4 2022-06-01 00:04:52            66  51.90750   

   Longitude     Temp_C  DewPoint_C  U_Wind_10m  V_Wind_10m  MSL_Pressure_hPa  \
0    0.23500  16.471338   15.322229   -0.496323    1.753113         1023.8725   
1    0.23500  16.471338   15.322229   -0.496323    1.753113         1023.8725   
2    0.16056  16.471338   15.322229   -0.496323    1.753113         1023.8725   
3    0.09750  16.471338   15.322229   -0.496323    1.753113         1023.8725   
4    0.08195  16.471338   15.322229   -0.496323    1.753113         1023.8725   

   Surface_Pressure_hPa  CloudBase_m  LowCloud  To

In [12]:
import pandas as pd
import numpy as np
from math import radians, cos, sin, asin, sqrt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import matplotlib.pyplot as plt

# --- 1) Tarih sütunlarını datetime tipine çevirme ---
df_filed["Time Over"]  = pd.to_datetime(df_filed["Time Over"])
df_actual["Time Over"] = pd.to_datetime(df_actual["Time Over"])
# Eğer df_meta'da da tarih varsa:
df_meta["ACTUAL OFF BLOCK TIME"]   = pd.to_datetime(df_meta["ACTUAL OFF BLOCK TIME"])

# --- 2) Kesme tarihi ile filtreleme ---
cutoff = pd.Timestamp("2022-06-07 23:00:00")
filed_cut  = df_filed[df_filed["Time Over"] <= cutoff]
actual_cut = df_actual[df_actual["Time Over"] <= cutoff]
meta_cut   = df_meta[df_meta["ACTUAL OFF BLOCK TIME"] <= cutoff]

# 3) Filtre sonrası unique ECTRL ID sayısını gösterme
unique_meta_ids = meta_cut["ECTRL ID"].nunique()
print(f"Filtre sonrası unique ECTRL ID sayısı: {unique_meta_ids}")

# --- 4) Filed ve Actual verilerini birleştirme ---
merged = actual_cut.merge(
    filed_cut,
    on=["ECTRL ID", "Sequence Number"],
    suffixes=("_actual", "_filed"),
    how="inner"
)


# --- 6) Sapma hesaplama (Haversine) ---
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1)*cos(lat2)*sin(dlon/2)**2
    return 6371 * 2 * asin(sqrt(a))

merged["deviation_km"] = merged.apply(
    lambda r: haversine(
        r["Latitude_actual"], r["Longitude_actual"],
        r["Latitude_filed"],   r["Longitude_filed"]
    ),
    axis=1
)

# --- 7) One-hot encoding ---
onehot = pd.get_dummies(merged[["ADEP","ADES"]], prefix=["ADEP","ADES"])
merged = pd.concat([merged, onehot], axis=1)

# --- 8) Özellik ve hedef tanımları ---
feature_cols = [
    "Latitude_filed", "Longitude_filed", "Flight Level_filed",
    # Meteo değişkenleri (filed zamanı)
    "Temp_C_filed", "DewPoint_C_filed", "U_Wind_10m_filed", "V_Wind_10m_filed",
    "MSL_Pressure_hPa_filed", "Surface_Pressure_hPa_filed",
    "CloudBase_m_filed", "LowCloud_filed", "TotalCloud_filed"
] + list(onehot.columns)

X = merged[feature_cols]
y = merged["deviation_km"]

# --- 9) Train/Test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- 10) Modelleri tanımlayıp eğitme ve karşılaştırma ---
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}
results = {}

for name, mdl in models.items():
    mdl.fit(X_train, y_train)
    preds = mdl.predict(X_test)
    results[name] = {
        "MAE": mean_absolute_error(y_test, preds),
        "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
        "R2": r2_score(y_test, preds)
    }
    # Scatter plot
    plt.figure(figsize=(6,5))
    plt.scatter(y_test, preds, alpha=0.5)
    lim = max(y_test.max(), preds.max())
    plt.plot([0, lim], [0, lim], 'r--')
    plt.title(f"{name}: Gerçek vs Tahmin Sapma")
    plt.xlabel("Gerçek Sapma (km)")
    plt.ylabel("Tahmin Sapma (km)")
    plt.grid(True)
    plt.show()

# Performansları tablo şeklinde gösterme
metrics_df = pd.DataFrame(results).T.rename_axis("Model").reset_index()
print(metrics_df)

# --- 11) En çok sapma yaratan 10 uçuşu gösterme ---
# Uçuş bazında maksimum sapmayı hesapla
flight_dev = merged.groupby("ECTRL ID")["deviation_km"].max().reset_index()
# En yüksek 10 sapma
top10 = flight_dev.nlargest(10, "deviation_km")
print("\nEn çok sapma yaratan 10 uçuş (ECTRL ID ve sapma km):")
print(top10)

Filtre sonrası unique ECTRL ID sayısı: 185550


ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [20]:
# 1) Zaman sıralaması (aynı uçuş içindeki akış bozulmasın)
merged.sort_values(["ECTRL ID", "Time Over_actual"], inplace=True)

# 2) Aynı ECTRL ID grubunda önceki ve sonraki geçerli değerlerle impute et
merged["CloudBase_m_filed"] = (
    merged
      .groupby("ECTRL ID")["CloudBase_m_filed"]
      .apply(lambda s: s.ffill().bfill())
      .reset_index(level=0, drop=True)
)

# 3) Son durumu kontrol et
print("CloudBase_m_filed hâlâ NaN sayısı:", merged["CloudBase_m_filed"].isna().sum())

# Artık merged DataFrame’in içinde CloudBase_m_filed eksiksiz güncellenmiş durumda.
# İsterseniz bu haliyle df_filed veya df_actual’a da geri dağıtabilirsiniz,
# ama genelde bu merged üzerinden devam etmek yeterli.


CloudBase_m_filed hâlâ NaN sayısı: 48454


In [21]:
# 4) Kalan NaN’ları 0 ile doldur
merged["CloudBase_m_filed"].fillna(0, inplace=True)

# 5) Kontrol
print("İşlem sonrası CloudBase_m_filed hâlâ NaN sayısı:", merged["CloudBase_m_filed"].isna().sum())


İşlem sonrası CloudBase_m_filed hâlâ NaN sayısı: 0


<ipython-input-21-19e3aaed91a9>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged["CloudBase_m_filed"].fillna(0, inplace=True)


In [ ]:
# --- 10) Modelleri tanımlayıp eğitme ve karşılaştırma ---
models = {
    "LinearRegression": LinearRegression(),
    "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
    "GradientBoosting": GradientBoostingRegressor(n_estimators=100, random_state=42)
}
results = {}

for name, mdl in models.items():
    mdl.fit(X_train, y_train)
    preds = mdl.predict(X_test)
    results[name] = {
        "MAE": mean_absolute_error(y_test, preds),
        "RMSE": np.sqrt(mean_squared_error(y_test, preds)),
        "R2": r2_score(y_test, preds)
    }
    # Scatter plot
    plt.figure(figsize=(6,5))
    plt.scatter(y_test, preds, alpha=0.5)
    lim = max(y_test.max(), preds.max())
    plt.plot([0, lim], [0, lim], 'r--')
    plt.title(f"{name}: Gerçek vs Tahmin Sapma")
    plt.xlabel("Gerçek Sapma (km)")
    plt.ylabel("Tahmin Sapma (km)")
    plt.grid(True)
    plt.show()

# Performansları tablo şeklinde gösterme
metrics_df = pd.DataFrame(results).T.rename_axis("Model").reset_index()
print(metrics_df)

# --- 11) En çok sapma yaratan 10 uçuşu gösterme ---
# Uçuş bazında maksimum sapmayı hesapla
flight_dev = merged.groupby("ECTRL ID")["deviation_km"].max().reset_index()
# En yüksek 10 sapma
top10 = flight_dev.nlargest(10, "deviation_km")
print("\nEn çok sapma yaratan 10 uçuş (ECTRL ID ve sapma km):")
print(top10)

In [18]:
# Her sütundaki eksik oranı (%)
missing_pct = merged.isna().mean() * 100

# Sadece eksik değeri olanları göster
print(missing_pct[missing_pct > 0])


CloudBase_m_actual    21.105962
CloudBase_m_filed     21.056479
ADEP Latitude          0.081460
ADEP Longitude         0.081460
ADES Latitude          0.093637
ADES Longitude         0.093637
AC Registration        0.045139
dtype: float64
